# Hypothesis Testing

Brittany Lassiter

# Imports

In [4]:
import numpy as np
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt

from urllib.parse import quote_plus
import pymysql
pymysql.install_as_MySQLdb()

from sqlalchemy import create_engine
from sqlalchemy_utils import create_database, database_exists

from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [5]:
## loading mysql credentials
with open('/Users/blass/.secret/mysql.json') as f:
    login = json.load(f)
# login.keys()

In [18]:
import glob
# Use glob to get all filepaths that match the pattern (*=wildcard)
tmdb_files = sorted(glob.glob("Data/final_tmdb_data*.csv.gz"))
tmdb_files

['Data\\final_tmdb_data_2001.csv.gz',
 'Data\\final_tmdb_data_2002.csv.gz',
 'Data\\final_tmdb_data_2010.csv.gz']

In [19]:
pd.read_sql("SHOW TABLES;", engine)

,Tables_in_movies
0,genres
1,ratings
2,title_basics
3,title_genres
4,tmdb_data


In [13]:
pd.set_option('display.max_columns',100)

In [14]:
# Use read_csv in a list comprehension and combine with concat to load all files
df = pd.concat([pd.read_csv(f) for f in tmdb_files] )
display(df.head())
df.info()

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0035423,0.0,/tJLV3BAlHOgscVOrA99Wnb2gAef.jpg,NaN,48000000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 14, ...",NaN,11232.0,en,Kate & Leopold,When her scientist ex-boyfriend discovers a po...,16.141,/mUvikzKJJSg9khrVdxK8kg3TMHA.jpg,"[{'id': 85, 'logo_path': None, 'name': 'Konrad...","[{'iso_3166_1': 'US', 'name': 'United States o...",2001-12-25,76019048.0,118.0,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,"If they lived in the same century, they'd be p...",Kate & Leopold,0.0,6.320,1237.0,PG-13
2,tt0114447,0.0,NaN,NaN,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",NaN,151007.0,en,The Silent Force,An elite Federal force whose objective is to e...,2.537,/u0njmfXD3dXxQfeykBHBuFQOYqD.jpg,[],[],2001-01-01,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,They left him for dead... They should have fin...,The Silent Force,0.0,5.000,3.0,NaN
3,tt0116916,0.0,/rFpHBidSlhjflmnLu7BZilyKeQR.jpg,NaN,0.0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,73549.0,en,The Dark Mist,With the future of the world hanging in the ba...,1.400,/z18E0lukzOtAALmfINSTCUU0Als.jpg,"[{'id': 71364, 'logo_path': None, 'name': 'Car...","[{'iso_3166_1': 'US', 'name': 'United States o...",1996-01-01,0.0,101.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,The Dark Mist,0.0,3.500,2.0,PG
4,tt0118589,0.0,/9NZAirJahVilTiDNCHLFcdkwkiy.jpg,NaN,22000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",NaN,10696.0,en,Glitter,"A young woman is catapulted into pop stardom, ...",7.593,/b8gKsXB6d60SC2HKUcAAzTacn0S.jpg,"[{'id': 89627, 'logo_path': None, 'name': 'Gli...","[{'iso_3166_1': 'US', 'name': 'United States o...",2001-09-21,5271666.0,104.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"In music she found her dream, her love, herself.",Glitter,0.0,4.405,132.0,PG-13


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2655 entries, 0 to 1
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb_id                2655 non-null   object 
 1   adult                  2652 non-null   float64
 2   backdrop_path          1523 non-null   object 
 3   belongs_to_collection  217 non-null    object 
 4   budget                 2652 non-null   float64
 5   genres                 2652 non-null   object 
 6   homepage               214 non-null    object 
 7   id                     2652 non-null   float64
 8   original_language      2652 non-null   object 
 9   original_title         2652 non-null   object 
 10  overview               2593 non-null   object 
 11  popularity             2652 non-null   float64
 12  poster_path            2411 non-null   object 
 13  production_companies   2652 non-null   object 
 14  production_countries   2652 non-null   object 
 15  release

In [7]:
# Setting Alpha
alpha = 0.5

# Questions to answers

***Q1: Does the MPAA rating of a movie (G/PG/PG-13/R) affect how much revenue the movie generates?***

Null Hypothesis

There is no difference of the revenue based on the movie rating.

Alternate Hypothesis 

The revenue has a significant amount generated due to the movie ratings.

**Testing to use**

We have Numerical Data
with more than 2 groups
We want a One way ANOVA
ANOVA Assumptions

## Cleaning Data

In [23]:
engine = create_engine('Data/final_tmdb_data*.csv.gz')

ArgumentError: Could not parse SQLAlchemy URL from string 'Data/final_tmdb_data*.csv.gz'

In [21]:
# Creating dataframe
q = """
SELECT revenue, certification
FROM tmdb_data
"""
mpaa_df = pd.read_sql(q,engine)
mpaa_df.head()

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'revenue' in 'field list'")
[SQL: 
SELECT revenue, certification
FROM tmdb_data
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [17]:
df["certification"].value_counts()

R        491
PG-13    201
NR        81
PG        68
G         22
NC-17      6
Name: certification, dtype: int64